In [1]:
from __future__ import print_function

import numpy as np
import pandas as pd
np.random.seed(1)

import sys
import sklearn
import sklearn.ensemble
import sklearn.cluster
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

%load_ext autoreload
%autoreload 2

import shap
from anchor import utils
from int_met import *

C:\Users\youss\AppData\Local\conda\conda\envs\deeplearning\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
dataset_folder = 'Datasets/'
dataset = utils.load_dataset('mortality', discretize=False, balance=True, dataset_folder=dataset_folder)

In [3]:
X_train = pd.DataFrame(data=dataset.train, columns=dataset.feature_names)
y_train = dataset.labels_train
X_test = pd.DataFrame(data=dataset.test, columns=dataset.feature_names)
y_test = dataset.labels_test

In [4]:
c = sklearn.ensemble.RandomForestClassifier(n_estimators=70, n_jobs=5, random_state=1)
c.fit(X_train, y_train)
predict_fn = lambda x: c.predict(x)
print('Train', sklearn.metrics.accuracy_score(y_train, predict_fn(X_train)))
print('Test', sklearn.metrics.accuracy_score(y_test, predict_fn(X_test)))

Train 1.0
Test 0.8504784688995215


In [5]:
explainer = shap.KernelExplainer(predict_fn, X_train.median())

In [6]:
exp = explainer.shap_values(X_test[0:400])
exp2 = explainer.shap_values(X_test[0:400])

100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [01:30<00:00,  4.42it/s]


In [7]:
i = calc_identity(exp, exp2)
print(i)

(0.0, 400, 400)


In [8]:
print(calc_stability(exp, y_test[0:400]))

C:\Users\youss\AppData\Local\conda\conda\envs\deeplearning\lib\site-packages\sklearn\cluster\k_means_.py:896: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)


(102, 400)


In [9]:
s = calc_separability(X_test[0:400], explainer.shap_values)
print(s)

100%|████████████████████████████████████████████████████████████████████████████████| 390/390 [01:27<00:00,  4.63it/s]


(3446, 152100, 2.2656147271531886)
